# Speeding up The Z function

So the main time consumed to solve the electrostatic function is actually the evaluation of the function itself !

We will try to improve it's evaluation time


In [1]:
#import the functions
from scipy.special import wofz
import numpy as np
np.random.seed()

import matplotlib as plt
%load_ext line_profiler

In [2]:
#total size, for the cache estimation
from sys import getsizeof, stderr
from itertools import chain
from collections import deque
try:
    from reprlib import repr
except ImportError:
    pass

def total_size(o, handlers={}, verbose=False):
    """ Returns the approximate memory footprint an object and all of its contents.

    Automatically finds the contents of the following builtin containers and
    their subclasses:  tuple, list, deque, dict, set and frozenset.
    To search other containers, add handlers to iterate over their contents:

        handlers = {SomeContainerClass: iter,
                    OtherContainerClass: OtherContainerClass.get_elements}

    """
    dict_handler = lambda d: chain.from_iterable(d.items())
    all_handlers = {tuple: iter,
                    list: iter,
                    deque: iter,
                    dict: dict_handler,
                    set: iter,
                    frozenset: iter,
                   }
    all_handlers.update(handlers)     # user handlers take precedence
    seen = set()                      # track which object id's have already been seen
    default_size = getsizeof(0)       # estimate sizeof object without __sizeof__

    def sizeof(o):
        if id(o) in seen:       # do not double count the same object
            return 0
        seen.add(id(o))
        s = getsizeof(o, default_size)

        if verbose:
            print(s, type(o), repr(o), file=stderr)

        for typ, handler in all_handlers.items():
            if isinstance(o, typ):
                s += sum(map(sizeof, handler(o)))
                break
        return s

    return sizeof(o)

In [3]:
N_cases = 10000
w = np.random.uniform(-10, 10, N_cases) + 1.j * np.random.uniform(-10, 10, N_cases)

In [4]:
%%timeit
z_w = wofz(w)

2.68 ms ± 205 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


# Unvectorizing it

Ok, so Alexis think that we can speed up the function using a cache.
Unfortunatly, the cache do not work with numpy array

So wee will check if un-vectorizing the call can allow us to gain from the cache

In [5]:
def unvect_wofz(ws):
    z = np.zeros(len(ws), dtype="complex128")
    for i,w in enumerate(ws):
        z[i] = wofz(w)
    return z

In [6]:
%%timeit
z_w = unvect_wofz(w)

18.5 ms ± 1.02 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [7]:
a = %lprun -r -f unvect_wofz z_w = unvect_wofz(w)
a.print_stats()

Timer unit: 1e-06 s

Total time: 0.027457 s
File: <ipython-input-5-28e2f432af96>
Function: unvect_wofz at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def unvect_wofz(ws):
     2         1        137.0    137.0      0.5      z = np.zeros(len(ws), dtype="complex128")
     3     10001       5756.0      0.6     21.0      for i,w in enumerate(ws):
     4     10000      21563.0      2.2     78.5          z[i] = wofz(w)
     5         1          1.0      1.0      0.0      return z



Well, even if we can gain a factor 10 with the cache, we loose a factor 10 with the unfectorasition


# With Cython

In [8]:
%load_ext Cython

In [9]:
z = np.ndarray(N_cases, dtype=np.complex128)

In [10]:
%%cython -f -c=-O3 

import numpy as np
cimport numpy as np

from scipy.special.cython_special cimport wofz as cwofz


DTYPE_c = np.complex128
ctypedef np.complex128_t DTYPE_c_t
cy_dict = {}
# @cython.boundscheck(False) # turn off bounds-checking for entire function
# @cython.wraparound(False)  # turn off negative index wrapping for entire function
def cythonwrapper_wofz(np.ndarray[DTYPE_c_t, ndim=1] w  not None, np.ndarray[DTYPE_c_t, ndim=1] output not None):
    assert w.dtype == DTYPE_c and output.dtype == DTYPE_c
    cython_wofz(w.shape[0], &w[0], &output[0] )

cdef cython_wofz(int n, DTYPE_c_t *w, DTYPE_c_t *output):
    cdef int i
    for i in range(n):
        output[i] = cwofz(w[i]) 
    
    
def cythonwrapper_wofz_cached(np.ndarray[DTYPE_c_t, ndim=1] w  not None, np.ndarray[DTYPE_c_t, ndim=1] output not None):
    assert w.dtype == DTYPE_c and output.dtype == DTYPE_c
    cython_wofz_cached(w.shape[0], &w[0], &output[0] )

cdef cython_wofz_cached(int n, DTYPE_c_t *w, DTYPE_c_t *output):
    cdef int i
    for i in range(n):
        if w[i] in cy_dict:
            output[i] = cy_dict[w[i]]
        else:
            output[i] = cwofz(w[i]) 
            cy_dict.update({w[i]:output[i]})

In [11]:
%%timeit 
for i in range(1000):
    wofz(w)

2.38 s ± 84.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [12]:
%%timeit 
for i in range(1000):
    cythonwrapper_wofz(w, z)

2.33 s ± 46.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [13]:
%%timeit 
for i in range(1000):
    cythonwrapper_wofz_cached(w, z)

2.14 s ± 82.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


So the `cython` `for` loop, when well designed, is **as fast as the numpy call** !

Using a cache (a dictionnary) we do found a speed up of 

## Let's profile it

we will used a line profiler to see what take the most of the time

In [14]:
import Cython
import line_profiler
directive_defaults =  Cython.Compiler.Options.get_directive_defaults()

directive_defaults['linetrace'] = True
directive_defaults['binding'] = True

In [15]:
%%cython -f --compile-args=-DCYTHON_TRACE=1 -c=-O3 

import numpy as np
cimport numpy as np

from scipy.special.cython_special cimport wofz as cwofz

DTYPE_c = np.complex128
ctypedef np.complex128_t DTYPE_c_t
cache_dict = {}

# @cython.boundscheck(False) # turn off bounds-checking for entire function
# @cython.wraparound(False)  # turn off negative index wrapping for entire function
def cython_wofz(np.ndarray[DTYPE_c_t, ndim=1] w  not None, np.ndarray[DTYPE_c_t, ndim=1] output not None):
    assert w.dtype == DTYPE_c and output.dtype == DTYPE_c
    cdef int i
    for i in range(w.shape[0]):
        output[i] = cwofz(w[i])     

    
def cython_wofz_cached(np.ndarray[DTYPE_c_t, ndim=1] w  not None,
                       np.ndarray[DTYPE_c_t, ndim=1] output not None,
                      dict cache_dict):
    assert w.dtype == DTYPE_c and output.dtype == DTYPE_c
    cdef int i
    for i in range(w.shape[0]):
        if w[i] in cache_dict:
            output[i] = cache_dict[w[i]]
        else:
            output[i] = cwofz(w[i]) 
            cache_dict.update({w[i]:output[i]})
    


In [16]:
%time cython_wofz(w, z)
cache_dict = {}
%time cython_wofz_cached(w, z, cache_dict)
%time cython_wofz_cached(w, z, cache_dict) 


CPU times: user 2.98 ms, sys: 0 ns, total: 2.98 ms
Wall time: 2.96 ms
CPU times: user 9.04 ms, sys: 0 ns, total: 9.04 ms
Wall time: 9.09 ms
CPU times: user 2.46 ms, sys: 0 ns, total: 2.46 ms
Wall time: 2.49 ms


In [17]:
a = %lprun -r -f cython_wofz cython_wofz(w, z)
a.print_stats()

Timer unit: 1e-06 s

Total time: 0.006353 s
File: /home/tavant/.cache/ipython/cython/_cython_magic_16fee34bdd85031f9999a1d8abf9aa2f.pyx
Function: cython_wofz at line 13

Line #      Hits         Time  Per Hit   % Time  Line Contents
    13                                           def cython_wofz(np.ndarray[DTYPE_c_t, ndim=1] w  not None, np.ndarray[DTYPE_c_t, ndim=1] output not None):
    14         1          4.0      4.0      0.1      assert w.dtype == DTYPE_c and output.dtype == DTYPE_c
    15                                               cdef int i
    16         1          1.0      1.0      0.0      for i in range(w.shape[0]):
    17     10000       6348.0      0.6     99.9          output[i] = cwofz(w[i])     



In [18]:
cache_dict = {}
print(f"size of the cache: {len(cache_dict)} elements, equivalent to a memory usage of {total_size(cache_dict):2.2e} bytes")

a = %lprun -r -f cython_wofz_cached cython_wofz_cached(w, z, cache_dict)
a.print_stats()
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
print(f"size of the cache: {len(cache_dict)} elements, equivalent to a memory usage of {total_size(cache_dict):2.2e} bytes")
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
a = %lprun -r -f cython_wofz_cached cython_wofz_cached(w, z, cache_dict)
a.print_stats()

size of the cache: 0 elements, equivalent to a memory usage of 2.40e+02 bytes
Timer unit: 1e-06 s

Total time: 0.017069 s
File: /home/tavant/.cache/ipython/cython/_cython_magic_16fee34bdd85031f9999a1d8abf9aa2f.pyx
Function: cython_wofz_cached at line 20

Line #      Hits         Time  Per Hit   % Time  Line Contents
    20                                           def cython_wofz_cached(np.ndarray[DTYPE_c_t, ndim=1] w  not None,
    21                                                                  np.ndarray[DTYPE_c_t, ndim=1] output not None,
    22                                                                 dict cache_dict):
    23         1          3.0      3.0      0.0      assert w.dtype == DTYPE_c and output.dtype == DTYPE_c
    24                                               cdef int i
    25         1          0.0      0.0      0.0      for i in range(w.shape[0]):
    26     10000       4529.0      0.5     26.5          if w[i] in cache_dict:
    27                     

In [19]:
%%cython -f -c=-O3 

import numpy as np
cimport numpy as np

from scipy.special import wofz
from scipy.special.cython_special cimport wofz as cwofz

DTYPE_c = np.complex128
ctypedef np.complex128_t DTYPE_c_t

def memodict(f):
    """ Memoization decorator for a function taking a single argument """
    class memodict(dict):
        def __missing__(self, key):
            ret = self[key] = f(key)
            return ret 
    return memodict().__getitem__

cached_cwofz = memodict(cwofz)
# @cython.boundscheck(False) # turn off bounds-checking for entire function
# @cython.wraparound(False)  # turn off negative index wrapping for entire function
def cython_wofz2(np.ndarray[DTYPE_c_t, ndim=1] w  not None, np.ndarray[DTYPE_c_t, ndim=1] output not None):
    assert w.dtype == DTYPE_c and output.dtype == DTYPE_c
    cdef int i
    for i in range(w.shape[0]):
        output[i] = cached_cwofz(w[i])     



In [20]:
%time cython_wofz2(w, z)
%time cython_wofz2(w, z)

CPU times: user 21.3 ms, sys: 1.99 ms, total: 23.3 ms
Wall time: 23.4 ms
CPU times: user 2.39 ms, sys: 0 ns, total: 2.39 ms
Wall time: 2.42 ms


In [21]:
def memodict(f):
    """ Memoization decorator for a function taking a single argument """
    class memodict(dict):
        def __missing__(self, key):
            ret = self[key] = f(key)
            return ret 
    return memodict().__getitem__

In [22]:
%%cython -f -c=-O3 

import numpy as np
cimport numpy as np

from scipy.special import wofz
from scipy.special.cython_special cimport wofz as cwofz

DTYPE_c = np.complex128
ctypedef np.complex128_t DTYPE_c_t

def memoize(f):
    class memodict(dict):
        __slots__ = ()
        def __missing__(self, key):
            self[key] = ret = f(key)
            return ret
    return memodict().__getitem__
cached_cwofz = memoize(cwofz)
# @cython.boundscheck(False) # turn off bounds-checking for entire function
# @cython.wraparound(False)  # turn off negative index wrapping for entire function
def cython_wofz3(np.ndarray[DTYPE_c_t, ndim=1] w  not None, np.ndarray[DTYPE_c_t, ndim=1] output not None):
    assert w.dtype == DTYPE_c and output.dtype == DTYPE_c
    cdef int i
    for i in range(w.shape[0]):
        output[i] = cached_cwofz(w[i])     



In [23]:
%time cython_wofz3(w, z)
%time cython_wofz3(w, z)

CPU times: user 21.4 ms, sys: 0 ns, total: 21.4 ms
Wall time: 22.8 ms
CPU times: user 1.59 ms, sys: 0 ns, total: 1.59 ms
Wall time: 1.61 ms
